In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming daily_stats and categories are already defined from previous analysis
# Define category names for the categories
category_names = {
    1: "No Trees",
    2: "Low Density",
    3: "Medium Density",
    4: "High Density"
}

# Prepare data for the DataFrame
# Replace 'categories' with your own list of category identifiers
cat_names = [category_names.get(cat, f"Unknown Category {cat}") for cat in categories]
mean_ranges = []
mean_stds = []
mean_temps = []
max_temps = []
min_temps = []

# Collect mean ranges, standard deviations, and temperature statistics for valid categories
for cat in categories:
    if cat in daily_stats:
        mean_ranges.append(daily_stats[cat]['range'].mean().item())
        mean_stds.append(daily_stats[cat]['std_dev'].mean().item())
        mean_temps.append(daily_stats[cat]['mean'].mean().item())
        max_temps.append(daily_stats[cat]['max'].mean().item())
        min_temps.append(daily_stats[cat]['min'].mean().item())

# Create a DataFrame to store our results
results = pd.DataFrame({
    'Category': cat_names,
    'Density_Order': categories,
    'Mean_Range': mean_ranges,
    'Mean_StdDev': mean_stds,
    'Mean_Temp': mean_temps,
    'Max_Temp': max_temps,
    'Min_Temp': min_temps
})

# Sort the results by tree density
results = results.sort_values('Density_Order')

# Calculate the temperature stability index
# This index combines the inverse of range and standard deviation
results['Temp_Stability_Index'] = (1 / results['Mean_Range']) + (1 / results['Mean_StdDev'])

# Normalize the index for easier interpretation
results['Temp_Stability_Index_Normalized'] = (results['Temp_Stability_Index'] - results['Temp_Stability_Index'].min()) / (results['Temp_Stability_Index'].max() - results['Temp_Stability_Index'].min())

# Calculate the correlation between tree density and temperature stability
correlation, p_value = stats.spearmanr(results['Density_Order'], results['Temp_Stability_Index_Normalized'])

# Print results
print(results)
print(f"\nCorrelation between Tree Density and Temperature Stability: {correlation:.3f}")
print(f"P-value: {p_value:.3f}")

# Calculate percentage increase in stability from lowest to highest density
stability_increase = results['Temp_Stability_Index_Normalized'].iloc[-1] - results['Temp_Stability_Index_Normalized'].iloc[0]
print(f"\nAbsolute increase in temperature stability from lowest to highest tree density: {stability_increase:.2f}")

# Analyze the rate of change in stability
results['Stability_Change'] = results['Temp_Stability_Index_Normalized'].diff()
results['Stability_Change_Rate'] = results['Stability_Change'] / results['Temp_Stability_Index_Normalized'].shift(1) * 100
results['Stability_Change_Rate'] = results['Stability_Change_Rate'].fillna(0)  # Replace NaN with 0 for the first row

print("\nRate of change in temperature stability between categories:")
print(results[['Category', 'Stability_Change_Rate']])

# Plotting
plt.figure(figsize=(12, 6))

# Create a scatter plot with color coding for categories
sns.scatterplot(data=results, x='Density_Order', y='Temp_Stability_Index_Normalized', 
                s=100, hue='Category', palette='viridis', legend='full')

# Add a regression line
sns.regplot(data=results, x='Density_Order', y='Temp_Stability_Index_Normalized', 
            scatter=False, color='red')

# Set plot titles and labels
plt.title('Temperature Stability Index vs Tree Density', fontsize=16)
plt.xlabel('Tree Density Category', fontsize=12)
plt.ylabel('Normalized Temperature Stability Index', fontsize=12)

# Set x-ticks to show category names
plt.xticks(results['Density_Order'], results['Category'], rotation=45, ha='right')

# Show legend
plt.legend(title='Tree Density Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout
plt.tight_layout()
plt.show()